In [1]:
import asyncio

from scipy import optimize

from tq42.channel import Channel, Ask, Tell
from tq42.client import TQ42Client
from tq42.algorithm import AlgorithmProto
from tq42.compute import HardwareProto
import numpy as np
import OptimizationTestFunctions as otf

from tq42.experiment_run import ExperimentRun
from tq42.algorithm import CvaOptParametersProto

In [ ]:
async def main():
    with TQ42Client() as client:
        client.login()
        channel = await Channel.create(client=client)
        channel_local = await Channel.create(client=client)

        tetra_opt_params = {
            "dimensionality": 2,
            "iteration_number": 2,
            "maximal_rank": 4,
            "points_number": 1,
            "quantization": False,
            "tolerance": 0.0010000000474974513,
            "lower_limits": [0, 0],
            "upper_limits": [9, 9],
            "grid": [10, 10],
             "objective_function_channel_id": channel.id,
            "local_optimizer_channel_id": channel_local.id,
            "polling": {"initial_delay": 1.0, "retries": 100, "delay": 1.0, "backoff_factor": 1.1}
        }

        run = ExperimentRun.create(
            client=client,
            algorithm=AlgorithmProto.TETRA_OPT,
            experiment_id="0ba18e6f-65e6-4c0a-bda3-091c5a45312d", 
            compute=HardwareProto.SMALL,
            parameters={'parameters': tetra_opt_params, 'inputs': {}}
        )

        print(f"Experiment run id: {run.id}")

        async def objective_function_callback(ask: Ask) -> Tell:
            print("obj func len(ask.headers): ", len(ask.headers))
            dim = len(ask.headers)
            func = otf.Ackley(dim)
            y = []
            for parameter in ask.parameters:
                print("obj func ask parameter: ", parameter.values)
                y.append(float(func(np.array(parameter.values))))
            # add result to data

            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=y
            )
            return tell

        async def local_optimization_function_callback(ask: Ask) -> Tell:
            dim = len(ask.headers)
            func = otf.Ackley(dim)
            y = []
            new_x = []
            for parameter in ask.parameters:
                res = optimize.minimize(func, np.array(parameter.values))
                new_x.append({"values": res.x})
                y.append(float(res.fun))

            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=y,
                candidates=new_x
            )
            return tell

        def success():
            print("Experiment done!")

        await asyncio.gather(
            channel.connect(
                 callback=objective_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            ),
            channel_local.connect(
                callback=local_optimization_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            )
        )

# Run the async function in the Jupyter Notebook
await main()


{"device_code":"2Ou--NlJlgxrTp2rdrCmxU1I","user_code":"GBKP-VBBZ","verification_uri":"https://auth.terraquantum.io/activate","expires_in":900,"interval":5,"verification_uri_complete":"https://auth.terraquantum.io/activate?user_code=GBKP-VBBZ"}
If a browser does not open, please access this URL: https://auth.terraquantum.io/activate?user_code=GBKP-VBBZ to login. Also check the code: GBKP-VBBZ
Authentication is successful, access token is saved in: keyring.
org:2b040bcb-1697-491f-83d8-9da045c93eaf
proj:347826c3-5b62-4bd7-ad06-cfd11b9e4746
Experiment run id: 9078a907-d6af-43ec-8c0a-1c358684b2a1
obj func len(ask.headers):  2
obj func ask parameter:  [1.0, 3.0]
obj func ask parameter:  [2.0, 7.0]
obj func ask parameter:  [3.0, 0.0]
obj func ask parameter:  [4.0, 5.0]
obj func ask parameter:  [5.0, 6.0]
obj func ask parameter:  [6.0, 2.0]
obj func ask parameter:  [7.0, 8.0]
obj func ask parameter:  [8.0, 4.0]
obj func len(ask.headers):  2
obj func ask parameter:  [0.0, 6.0]
obj func ask para